# Tutorial 5 - Advanced quantum programming

Note: This tutorial is designed for users who already understand quantum programming using the `QoolQit` library which uses the `pulser` library as a dependency. For more details on `QoolQit`, see [its tutorial](https://pasqal-io.github.io/qoolqit/latest/). For more details on `pulser`, see [its tutorial](https://pulser.readthedocs.io).

By design, a quantum solver is composed of two main quantum components for solving a problem:

- an embedder,  i.e. a mechanism used to customize the layout of atoms on the quantum device.
- a drive shaper, i.e. a mechanism used to customize the drive Hamiltonian to which the neutral atoms are subjected during the execution of the quantum algorithm.


These can be specified in the `SolverConfig`. Most users can use the default embedders and drive shapers provided with this library. However, if you wish to alter the behavior of the solver, perhaps for the sake of research, debugging or learning, you can implement your own embedders or drive shapers. This tutorial dives into currently available components, as well as designing custom ones.

## Embedder

### Default

When instantiating a `SolverConfig`, a default embedder is already made available. To access the resulting embedding from a solver, simply call the `embedding` method as follows:

In [ ]:
from mis import MISSolver, MISInstance, SolverConfig, LocalEmulator


from networkx import erdos_renyi_graph

# User can fix the seed for reproducibility 
seed = 0
graph = erdos_renyi_graph(n=25, p=0.4, seed=seed)
instance = MISInstance(graph)

qutip_config = LocalEmulator()

config = SolverConfig(backend = qutip_config)
solver = MISSolver(instance, config)


geometry = solver.embedding()

# draw the register
# geometry.draw()

### Custom

To design our own embedding method, we need to define a class inhereting from `mis.pipeline.embedder.BaseEmbedder` and implement an `embed` method as follows: 

In [ ]:
from mis.pipeline.embedder import BaseEmbedder
from qoolqit import Register
from mis.pipeline.layout import Layout

class DefaultEmbedder(BaseEmbedder):
    """ The DefaultEmbedder class available in mis.
    """

    def embed(self, instance: MISInstance, config: SolverConfig) -> Register:
        device = config.device
        assert device is not None

        # Use Layout helper to get rescaled coordinates and interaction graph
        layout = Layout.from_device(data=instance, device=device._device)

        # Finally, prepare register.
        conversion_factor = device.converter.factors[2]
        return Register(
            qubits={f"q{node}": pos / conversion_factor for (node, pos) in layout.coords.items()}
        )

custom_config = SolverConfig(backend = qutip_config, embedder=DefaultEmbedder())

In [ ]:
custom_solver = MISSolver(instance, custom_config)
default_geometry = custom_solver.embedding()

# draw the register
default_geometry.draw()

## Drive shaper

### Default

When instantiating a `SolverConfig`, a default drive shaper is already made available. To access the resulting drive from a solver, simply call the `drive` method as follows:

In [ ]:
default_drive = solver.drive(solver.embedding())
default_drive

### Custom

To design our own pulse shaping method, we need to define a class inhereting from `mis.pipeline.pulse.BasePulseShaper` and implement a `pulse` method and a `detuning` method as follows: 

In [ ]:
import numpy as np

from mis.pipeline.drive import BaseDriveShaper, DriveParameters
from qoolqit import Drive, Constant, WeightedDetuning

class ConstantDriveShaper(BaseDriveShaper):
    """
    We simply return a constant pulse.
    """

    def drive(self, config: SolverConfig, register: Register, instance: MISInstance) -> Drive:
        TIME, _, _ = config.device.converter.factors
        # for users more used to pulser, a conversion should be made to match QoolQit
        return Constant(duration=1000 / TIME, value=np.pi)
    
    def weighted_detuning(
        self,
        config: SolverConfig,
        register: Register,
        instance: MISInstance,
        parameters: DriveParameters,
    ) -> list[WeightedDetuning] | None:
        return None

In [ ]:
constant_drive_config = SolverConfig(backend = qutip_config, drive_shaper=ConstantDriveShaper())
constant_drive_solver = MISSolver(instance, constant_drive_config)
constant_drive = constant_drive_solver.pulse(constant_drive_solver.embedding())
constant_drive